In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("salaries.db")
columns = ["File_Date",
           "SubAgency", 
           "Age_Range",
           "Education_Level",
           "Pay_Plan",
           "Grade",
           "LOS_Level",
           "Occupation",
           "PATCO",
           "Adjusted_Basic_Pay",
           "Supervisory_Status",
           "TOA",
           "Work_Schedule",
           "NSFTP_Indicator"]

In [3]:
pd.read_sql_query("""
SELECT 
    File_Date, 
    Occupation,
    AVG(Adjusted_Basic_Pay) AS Average_Salary
FROM 
    data 
GROUP BY
    File_Date, 
    Occupation
;""", conn)

,File_Date,Occupation,Average_Salary
0,197309,0512,17625.546445
1,197309,1169,14960.371925
2,197309,1825,22933.340152
3,197309,1863,11555.353139
4,197309,2152,18238.713101
5,197312,0512,18209.805444
6,197312,1169,15635.160194
7,197312,1825,24119.018841
8,197312,1863,11728.762582
9,197312,2152,19266.824512


In [4]:
conn.close()